In [46]:
import os
import polars as pl
import requests
import time
import plotly.graph_objects as go

from binance.client import Client
from dotenv import load_dotenv

In [47]:
client = Client()
symbol = "BTCUSDT"
interval = Client.KLINE_INTERVAL_1MINUTE
limit = 1000

In [48]:
klines = client.get_klines(
    symbol=symbol,
    interval=interval,
    limit=limit
)

In [49]:
df = pl.DataFrame(klines, schema=[
    ("open_time", pl.Int64),
    ("open", pl.Float64),
    ("high", pl.Float64),
    ("low", pl.Float64),
    ("close", pl.Float64),      
    ("volume", pl.Float64),
    ("close_time", pl.Int64),
    ("quote_asset_volume", pl.Float64),
    ("number_of_trades", pl.Int64),
    ("taker_buy_base_asset_volume", pl.Float64),
    ("taker_buy_quote_asset_volume", pl.Float64),
    ("ignore", pl.Float64),
], orient="row").with_columns([
    pl.from_epoch("open_time", time_unit="ms"),
    pl.from_epoch("close_time", time_unit="ms"),
])

In [50]:
df.null_count()

open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
df.head(5)

open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
datetime[ms],f64,f64,f64,f64,f64,datetime[ms],f64,i64,f64,f64,f64
2025-11-20 12:26:00,91721.48,91729.77,91688.0,91696.52,7.99753,2025-11-20 12:26:59.999,733424.690397,2808,1.57315,144268.728843,0.0
2025-11-20 12:27:00,91696.52,91781.34,91696.06,91738.89,9.76061,2025-11-20 12:27:59.999,895473.796918,4284,4.78536,438912.250173,0.0
2025-11-20 12:28:00,91738.9,91758.19,91738.89,91747.58,7.48629,2025-11-20 12:28:59.999,686827.050999,1553,6.01729,552046.747473,0.0
2025-11-20 12:29:00,91747.59,91747.59,91726.47,91733.65,3.38019,2025-11-20 12:29:59.999,310075.523588,805,1.1768,107947.076677,0.0
2025-11-20 12:30:00,91733.64,91733.65,91722.72,91724.01,6.40977,2025-11-20 12:30:59.999,587931.702224,938,0.87227,80008.025598,0.0


In [52]:
dfp = df.to_pandas()

In [53]:
fig = go.Figure(
    data=[
        go.Candlestick(
            x=dfp["open_time"],
            open=dfp["open"],
            high=dfp["high"],
            low=dfp["low"],
            close=dfp["close"],
            increasing_line_color='green',
            decreasing_line_color='red'
        )
    ]
)

fig.show()

In [54]:
client = Client()
data = client.get_aggregate_trades(
    symbol=symbol,
    limit=limit
)

aggtrades = pl.DataFrame(data).rename({
        "a": "agg_trade_id",
        "p": "price",
        "q": "quantity",
        "f": "first_trade_id",
        "l": "last_trade_id",
        "T": "timestamp",
        "m": "is_buyer_maker",
        "M": "is_best_match"
    })

In [55]:
aggtrades.head(5)

agg_trade_id,price,quantity,first_trade_id,last_trade_id,timestamp,is_buyer_maker,is_best_match
i64,str,str,i64,i64,i64,bool,bool
3754699308,"""86097.14000000""","""0.00007000""",5536954377,5536954377,1763701447381,false,true
3754699309,"""86097.78000000""","""0.00006000""",5536954378,5536954378,1763701447381,false,true
3754699310,"""86097.84000000""","""0.00047000""",5536954379,5536954379,1763701447381,false,true
3754699311,"""86097.90000000""","""0.00021000""",5536954380,5536954382,1763701447381,false,true
3754699312,"""86097.91000000""","""0.00006000""",5536954383,5536954383,1763701447381,false,true


In [58]:
df = df.with_columns([
    #pl.from_epoch("open_time", unit="ms").alias("open_time_dt"),
    #pl.from_epoch("close_time", unit="ms").alias("close_time_dt"),
    pl.from_epoch("open_time", time_unit="ms"),
    pl.from_epoch("close_time", time_unit="ms"),
])

print(df)


shape: (1_000, 12)
┌─────────────┬──────────┬──────────┬──────────┬───┬────────────┬────────────┬────────────┬────────┐
│ open_time   ┆ open     ┆ high     ┆ low      ┆ … ┆ number_of_ ┆ taker_buy_ ┆ taker_buy_ ┆ ignore │
│ ---         ┆ ---      ┆ ---      ┆ ---      ┆   ┆ trades     ┆ base_asset ┆ quote_asse ┆ ---    │
│ datetime[ms ┆ f64      ┆ f64      ┆ f64      ┆   ┆ ---        ┆ _volume    ┆ t_volume   ┆ f64    │
│ ]           ┆          ┆          ┆          ┆   ┆ i64        ┆ ---        ┆ ---        ┆        │
│             ┆          ┆          ┆          ┆   ┆            ┆ f64        ┆ f64        ┆        │
╞═════════════╪══════════╪══════════╪══════════╪═══╪════════════╪════════════╪════════════╪════════╡
│ 2025-11-20  ┆ 91721.48 ┆ 91729.77 ┆ 91688.0  ┆ … ┆ 2808       ┆ 1.57315    ┆ 144268.728 ┆ 0.0    │
│ 12:26:00    ┆          ┆          ┆          ┆   ┆            ┆            ┆ 843        ┆        │
│ 2025-11-20  ┆ 91696.52 ┆ 91781.34 ┆ 91696.06 ┆ … ┆ 4284       ┆ 4.7853